In [1]:
import numpy as np
import pandas as pd

In [110]:
from IPython.display import Image, display
from collections import Counter

In [109]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [5]:
import nltk
from nltk.corpus import stopwords

In [6]:
import spacy
from spacy import displacy
import eli5

In [7]:
import re
import warnings
warnings.filterwarnings("ignore")

## 1) немного структурируем данные

In [8]:
df_row = pd.read_csv("leto.csv")
df_row = df_row.rename(columns = {'Rating':'class', 'Content':'text'})
df_train = df_row.drop(columns=['Date'])

df_train["class"] = np.where(df_train["class"] >= 4, 1, 0)
df_test = df_train.iloc[0:4000]
df_test = df_test[['text', 'class']]
df_train = df_train[['text', 'class']]

df_train = df_train.dropna()
df_test = df_test.dropna()

df_train.tail()

,text,class
20654,"Ну и шляпа,с роот правами бесполезная прога,ра...",0
20655,Ок,1
20656,Доволен,1
20657,"Песопаснасть, рут ни нужын",0
20658,Сбербанк бомбовая компания на сегодняшний день...,1


## 2) с помощью библиотеки eli5 найдём тоновые                    слова

In [9]:
classifier = LogisticRegression()
vectorizer = CountVectorizer()
model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(df_train['text'], df_train['class'])

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression())])

In [10]:
eli5.show_weights(classifier, vec=vectorizer, top=40)

Weight?,Feature
+2.863,отличное
+2.756,удобно
+2.564,класс
+2.464,отлично
+2.365,супер
+2.345,хорошее
+2.305,удобное
+2.224,круто
+2.020,ок
+1.998,хорошо


## 3) некоторая обработка текста

In [11]:
tokenizer = nltk.tokenize.WordPunctTokenizer()
mystopwords = stopwords.words("russian")

def remove_punktuation(text):
    return re.sub(r'[^\w\s\d]', '', text)

def lower_case(text):
    text = str(text).lower()
    return ' '.join(tokenizer.tokenize(text))

def remove_stopwords(text):
    text = tokenizer.tokenize(text)
    text = [w for w in text if w not in mystopwords]
    return ' '.join(text)
    
def normalize(text):
    text = remove_punktuation(text)
    text = lower_case(text)
    text = remove_stopwords(text)
    return text

In [12]:
df_train['text'] = df_train['text'].apply(str)
df_test['text'] = df_test['text'].apply(str)

df_train['text'] = df_train['text'].apply(normalize)
df_test['text'] = df_test['text'].apply(normalize)

# 4) здесь не используем обучение

In [13]:
# Если ориентироваться на определённые слова (эмоциональные) и определять класс,
# сравнивая с тем как оно есть (по разметке), мы получим  ~71% попаданий.

positive_words = ('отличное', 'супер', 'класс', 'отлично', 'круто', 'удобно',
                  'отличный','классное', 'удобное', 'отличная', 'хорошее',
                  'нравится', 'довольна', 'ок', 'спасибо', 'пользуюсь', 'норм')


negative_words = ('отвратительно', 'дерьмо', 'удаляю', 'отстой', 'ужасно',
                   'ужас','отвратительное', 'пипец', 'вылетает', 'хреново', 'хрень',
                   'достал', 'полное', 'хуже')


positives_count = df_test.text.apply(lambda text: sum(word in text for word in positive_words))
negatives_count = df_test.text.apply(lambda text: sum(word in text for word in negative_words))

true_positive = (df_test['class']==1) == 1
is_positive = positives_count > negatives_count
correct_count = (is_positive == true_positive).values.sum()
print(f'correct count:{correct_count}, len(test_df):{len(df_test)}.')

accuracy = correct_count / len(df_test)
print('Test accuracy = {:.2%}'.format(accuracy))

correct count:2845, len(test_df):3998.
Test accuracy = 71.16%


### 5) CountVectorizer() + LogisticRegression()

In [14]:
model.fit(df_train['text'], df_train['class'])

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression())])

In [15]:
def eval_model(model, df_test):
    preds = model.predict(df_test['text'])
    print('Test accuracy = {:.2%}'.format(accuracy_score(df_test['class'], preds)))
    
eval_model(model, df_test)

Test accuracy = 96.10%


### 6) TfidfVectorizer() + LogisticRegression()

In [16]:
vectorizer = TfidfVectorizer()
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(df_train['text'], df_train['class'])

eval_model(model, df_test)

Test accuracy = 93.57%


## 7) использование NER

In [66]:
nlp = spacy.load('ru_core_news_lg', disable=['parser'])

docs = [doc for doc in nlp.pipe(df_train.text.values)]

In [75]:
for i in range(100):
    for token in docs[i]:
        print(token.text, token.ent_iob_, token.ent_type_)

it O 
just O 
works O 
целом O 
удобноное O 
приложениеиз O 
минусов O 
хотят O 
слишком O 
большой O 
доступ O 
персональным O 
данным O 
телефонеприходится O 
пользоваться O 
ограниченном O 
режиме O 
отлично O 
стал O 
зависать O 
1 O 
работы O 
антивируса O 
дальше O 
никуда O 
ранее O 
года O 
пользовался O 
нормально O 
очень O 
удобно O 
работает O 
быстро O 
всё O 
удобно O 
норм O 
очень O 
удобное O 
приложение O 
устраивает O 
работает O 
четко O 
отличии O 
банкоматов O 
которые O 
вечно O 
зависают O 
тупят O 
очень O 
ок O 
нормально O 
кроме O 
уведомление O 
удалять O 
стартует O 
доступа O 
gps O 
sms O 
звонкам O 
адресной O 
книге O 
филиал O 
фсб B ORG
одним O 
словом O 
очень O 
удобно O 
работает O 
замечательно O 
подвисаний O 
очень O 
удобно O 
очень O 
удобная O 
штука O 
отличное O 
приложение O 
удобно O 
практично O 
очень O 
удобное O 
приложение O 
оплаты O 
платежей O 
переводов O 
удобно O 
быстро O 
очень O 
удобное O 
приложение O 
транспортные O 
кар

In [ ]:
# Судя по всему NER здесь не поможет, разве что можно удалить ORG=сбербанк.

## 8) подготовка данных для сети

In [113]:
words_counter = Counter((word for text in df_train.text for word in text.lower().split()))

word2idx = {
    '': 0,
    '<unk>': 1
}
for word, count in words_counter.most_common():
    if count < 10:
        break
        
    word2idx[word] = len(word2idx)
    
print('Words count', len(word2idx))

Words count 1523


In [116]:
def convert(texts, word2idx, max_text_len):
    data = np.zeros((len(texts), max_text_len), dtype=np.int)
    
    for inx, text in enumerate(texts):
        result = []
        for word in text.split():
            if word in word2idx:
                result.append(word2idx[word])
        padding = [0]*(max_text_len - len(result))
        data[inx] = np.array(padding + result[-max_text_len:], dtype=np.int)
    return data

X_train = convert(df_train.text, word2idx, 1000)
X_test = convert(df_test.text, word2idx, 1000)

## 9) построение и обучение сети

In [117]:
model = Sequential([
    Embedding(input_dim=len(word2idx), output_dim=64, input_shape=(X_train.shape[1],)),
    GlobalMaxPooling1D(),
    Dense(units=10, activation='relu'),
    Dense(units=10, activation='relu'),
    
    Dense(units=1, activation='sigmoid')
])

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 1000, 64)          97472     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_13 (Dense)            (None, 10)                650       
                                                                 
 dense_14 (Dense)            (None, 10)                110       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 98,243
Trainable params: 98,243
Non-trainable params: 0
_________________________________________________

In [121]:
model.fit(X_train, 
          df_train['class'], 
          batch_size=128, 
          epochs=10, 
          validation_data=(X_test, df_test['class']))

Epoch 1/10
162/162 [==============================] - 6s 31ms/step - loss: 0.4271 - accuracy: 0.8081 - val_loss: 0.2321 - val_accuracy: 0.8414
Epoch 2/10
162/162 [==============================] - 5s 31ms/step - loss: 0.2427 - accuracy: 0.8618 - val_loss: 0.1898 - val_accuracy: 0.9355
Epoch 3/10
162/162 [==============================] - 5s 31ms/step - loss: 0.2031 - accuracy: 0.9265 - val_loss: 0.1613 - val_accuracy: 0.9427
Epoch 4/10
162/162 [==============================] - 5s 31ms/step - loss: 0.1774 - accuracy: 0.9360 - val_loss: 0.1415 - val_accuracy: 0.9527
Epoch 5/10
162/162 [==============================] - 5s 31ms/step - loss: 0.1596 - accuracy: 0.9445 - val_loss: 0.1298 - val_accuracy: 0.9572
Epoch 6/10
162/162 [==============================] - 5s 30ms/step - loss: 0.1450 - accuracy: 0.9519 - val_loss: 0.1150 - val_accuracy: 0.9665
Epoch 7/10
162/162 [==============================] - 5s 30ms/step - loss: 0.1326 - accuracy: 0.9586 - val_loss: 0.1039 - val_accuracy: 0.9687

In [123]:
model.evaluate(X_test, df_test['class'])

125/125 [==============================] - 0s 2ms/step - loss: 0.0822 - accuracy: 0.9782


[0.08218231052160263, 0.978239119052887]